# Generating Word Embeddings - Lab

## Introduction

In this lab, we'll learn how to generate our own word embeddings by training our own Word2Vec model, and also by building embedding layers right into our Deep Neural Networks!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, we'll start by creating our own word embeddings by making use of the Word2Vec Model. Then, we'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in our model. 

The easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which we can use to begin training immediately. For this lab, we'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.  In this lab, we'll learn how to train a Word2Vec model on the text data to generate word embeddings for them. In the next lab, we'll then use the vectors created by our Word2Vec model to effectively train a classifier to predict the category of news given the headline and description of each article. In this lab, we won't do any classification, although we will learn how to train a Word2Vec model and explore the relationships between different word vectors in our embedding!

Run the cell below to import everything we'll need for this lab. 

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, we'll import the data. You'll find the data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to also include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what our data looks like. 

In [3]:
import zipfile

zip_ref = zipfile.ZipFile('News_Category_Dataset_v2.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [9]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
raw_df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since we're working with text data, we'll still need to do some basic preprocessing and tokenize our data. You'll notice from the sample of the data above that two different columns contain text data--`headline` and `short_description`. The more text data our Word2Vec model has, the better it will perform. Therefore, we'll want to combine the two columns before tokenizing each comment and training our Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [11]:
raw_df['combined_text'] = raw_df['headline'] + ' ' + raw_df['short_description']
data = raw_df['combined_text'].map(word_tokenize)

Let's inspect the first 5 items in `data` to see how everything looks. 

In [12]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

You'll notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Recall from our previous lesson on how Word2Vec works that we can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If our window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example we saw).

Now that we've prepared our data, let's train our model and explore a bit!

## Training the Model

We'll start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [13]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that we've created our Word2Vec model, we still need to train it on our model. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which we can find in `model.corpus_count`. 
    * The number of `epochs` we want to train for, which we'll set to `10`

In [14]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55564817, 67352790)

Great! We now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which we'll find stored inside of `model.wv`. For simplicity's sake, let's go ahead and store this inside of the variable `wv` in order to save ourselves some keystrokes down the line. 

In [15]:
wv = model.wv

## Examining Our Word Vectors

Now that we have a trained Word2Vec model, let's go ahead and explore the relationships between some of the words in our corpus! 

One cool thing we can use Word2Vec for is to get the most similar words to a given word. We can do this passing in the word to `wv.most_similar()`. 

In the cell below, let's try getting the most similar word to `'Texas'`.

In [16]:
wv.most_similar('Texas')

[('Arkansas', 0.7997477054595947),
 ('Oregon', 0.791375994682312),
 ('California', 0.785560667514801),
 ('Massachusetts', 0.7852875590324402),
 ('Louisiana', 0.7844235897064209),
 ('Pennsylvania', 0.7843848466873169),
 ('Oklahoma', 0.7794086933135986),
 ('Florida', 0.777919352054596),
 ('Ohio', 0.7768120765686035),
 ('Utah', 0.774463415145874)]

Interesting! All of the most similar words are also states. 

We can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [18]:
wv.most_similar(negative='Texas')

[('Parent/Grandparent', 0.43031299114227295),
 ('much-vaunted', 0.4102620482444763),
 ('Un-Blind', 0.40927067399024963),
 ('Hunger-Free', 0.4031605124473572),
 ('uh-oh', 0.37930020689964294),
 ('UIUC', 0.37879127264022827),
 ('intercultural', 0.377349853515625),
 ('Trashbag', 0.376598596572876),
 ('Headstrong', 0.3715631067752838),
 ('Trump-Voting', 0.3625488877296448)]

These seem like just noise. This is because of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with our given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

We can also get the vector for a given word by passing in the word as if we were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [20]:
wv['Texas']

array([ 1.685373  ,  0.58286244, -0.93684715,  0.3521957 , -1.0105711 ,
       -1.0205977 , -1.1693631 , -1.9633377 ,  0.798597  , -0.06227722,
        0.49948785, -0.50811565,  0.19647765,  1.3769977 ,  0.65114677,
        2.3081965 ,  1.056608  ,  1.0624529 , -0.6955493 , -2.413092  ,
       -1.453819  ,  0.36710015,  1.8780468 ,  0.6249223 ,  0.25496212,
        2.01757   ,  0.46530354, -2.03028   , -2.2368262 , -0.539648  ,
        1.3306025 , -2.3058953 , -0.7507822 , -1.1240792 ,  0.5522561 ,
       -1.3039562 ,  1.2120734 , -0.7810352 ,  3.278555  , -2.0194492 ,
        0.01713911, -0.69421655,  0.07658327, -0.18797095,  1.0365833 ,
       -0.10361011,  0.68227583,  0.9653428 ,  0.50337714,  0.99199843,
        0.6383727 , -0.4346825 ,  0.48878646,  0.61002755, -0.10508267,
        1.1040783 , -0.18666194, -0.01193054, -0.215093  ,  0.43479976,
       -1.5589406 , -1.3623557 , -2.8795388 ,  0.8845144 ,  0.4184864 ,
        2.1295917 ,  0.65803075, -1.3354219 , -0.21342248, -3.24

Let's get all of the word vectors from the object at once. We can find these inside of `wv.vectors`.  Do this now in the cell below.  

In [21]:
wv.vectors

array([[-0.91578734, -0.08159476,  2.6983886 , ...,  2.1230426 ,
        -0.2271498 , -0.24774982],
       [-1.8455441 ,  0.6974033 ,  1.736803  , ...,  2.5297666 ,
        -1.2032583 , -0.45140824],
       [ 1.1878988 ,  0.06508322,  1.5627626 , ...,  0.573391  ,
        -1.0378884 ,  1.441136  ],
       ...,
       [-0.03643851,  0.04475371, -0.16860935, ..., -0.03191016,
         0.06028664,  0.06936393],
       [ 0.0378266 ,  0.0866734 , -0.06093905, ..., -0.0101607 ,
         0.03085637, -0.05667679],
       [-0.05766823,  0.06386673, -0.11223265, ..., -0.0033969 ,
        -0.01889998,  0.03810425]], dtype=float32)

As a final exercise, let's try recreating the _'king' - 'man' + 'woman' = 'queen'_ example we've seen before. We can do this by using the `most_similar` function and putting the things we want added together inside of an array passed to the `positive` parameter, and the things we want subtracted as an array passed to the the `negative` parameter. 

Do this now in the cell below. 

In [23]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.6213521957397461),
 ('revival', 0.6058666706085205),
 ('princess', 0.5932177305221558),
 ('crown', 0.5854739546775818),
 ('lover', 0.5746226906776428),
 ('reminiscent', 0.5732830762863159),
 ('brunette', 0.5716153383255005),
 ('symbol', 0.5679829120635986),
 ('spinoff', 0.5676065683364868),
 ('villain', 0.5673539042472839)]

As we can see from the output above, our model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As we can see from the word in first place, 'reminiscent', our model is far from perfect. This is likely because we didn't give it too much training, or training data. However, for the small amount of training data it was given, the model still performs remarkably well! 

We'll see in the next lab that from a practical standpoint, one of the best things we can do for performance is to start by loading in the weights from an open-sourced model that has been trained for a very long time on a massive amount of data, such as the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless our text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, we learned how to train and use a Word2Vec model to created vectorized word embeddings!